# Conexiones e importaciones

In [31]:
import os #trabajar con el sistema operativo y variables de entorno
from dotenv import load_dotenv #cargar variable de entorno
import spotipy #librería para trabajar con la API de Spotify
from spotipy.oauth2 import SpotifyClientCredentials #autenticación con Spotify
import pandas as pd #dataframes
import requests #peticiones a APIs
import time #librería para hacer pausas dentro de las peticiones y no saturar APIs
from urllib.parse import quote #esta librería sirve para codificar las URLs (espacios y caracteres especiales -> %20, etc)
import numpy as np
import mysql.connector
from mysql.connector import Error

In [25]:
load_dotenv() #carga las variables del entorno .env; devuelve un true o false

#Spotify
mis_credenciales = SpotifyClientCredentials(
    client_id=os.getenv("SPOTIFY_CLIENT_ID"),
    client_secret=os.getenv("SPOTIFY_CLIENT_SECRET")
)
spotify = spotipy.Spotify(auth_manager=mis_credenciales)

#last fm
api_key_lastfm = os.getenv("API_KEY_LASTFM")
shared_secret_lastfm = os.getenv("SHARED_SECRET_LASTFM")

#MySQL
MYSQL_HOST = os.getenv("MYSQL_HOST")
MYSQL_USER = os.getenv("MYSQL_USER")
MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD")

# Spotify

In [ ]:
def busqueda_spotify(genero, año):
    todas_las_canciones = []
    albumes_ya_vistos = set()  # Evita duplicados: un álbum puede aparecer al buscar varios artistas (ej. colaboraciones).
    datos_artistas = {}
    artistas_procesados = set()

    print (f"Buscando canciones de {año}\n")

    for tipo in genero:
        print("Género:", tipo)

        resultado_artistas = spotify.search(q="genre:" + genero, type="artist", limit=50)
        artistas = resultado_artistas["artists"]["items"]

        for artista in artistas:
            nombre_artista = artista["name"]

            #  Se buscan ámbumes del artista en el año informado
            busqueda = "artist:" + nombre_artista + " year:" + str(año)
            resultado_albumes = spotify.search(q=busqueda, type="album", limit=50)
            albumes = resultado_albumes["albums"]["items"]
            
            for album in albumes:
                id_album = album["id"]
                nombre_album = album["name"]
                
                # Igoramos álbumes ya vistos
                if id_album in albumes_ya_vistos:
                    continue
                
                #se añaden los álmbumes al set
                albumes_ya_vistos.add(id_album)

                # Y cogemos Pedimos las canciones del álbum
                resultado_canciones = spotify.album_tracks(id_album)
                canciones = resultado_canciones["items"]

                for cancion in canciones:
                    info = {
                        "nombre": cancion["name"],
                        "artista": nombre_artista,
                        "album": nombre_album,
                        "genero": genero,
                        "año": año
                    }
                    todas_las_canciones.append(info)

        print(f"Añadidas canciones de '{genero}'\n")
    

    # Contamos las canciones de cada género una por una
    for genero in generos:
        contador = 0
        for cancion in todas_las_canciones:
            if cancion["genero"] == genero:
                contador = contador + 1
        print(f"- {genero}: {contador} canciones")

    print(f"Canciones añadidas: {len(todas_las_canciones)}")
    todas_las_canciones_df = pd.DataFrame(todas_las_canciones)  
    
    return todas_las_canciones_df

SyntaxError: invalid syntax (3977237618.py, line 1)

In [ ]:
busqueda_spotify("rock", 2010)

NameError: name 'rock' is not defined

In [ ]:
def obtener_canciones_por_genero_y_año(genero, año):
    """
    Busca, mediante paginación, la mayor cantidad posible de canciones de Spotify 
    filtradas por género y año, hasta el límite de seguridad definido.
    Devuelve una lista de diccionarios con la información de las canciones.
    """
    
    busqueda_q = f"genre:{genero} year:{str(año)}" 
    lista_info_canciones = []
    offset = 0
    conteo_actual = 0
    
    # Bucle de Paginación: Continúa mientras haya menos canciones que el límite de seguridad
    while conteo_actual < MAX_CANCIONES_POR_GENERO:
        
        # Realizar la búsqueda con el offset y limit
        resultado_canciones = spotify.search(
            q=busqueda_q, 
            type="track", 
            limit=LIMIT_PER_REQUEST,
            offset=offset
        )
        
        canciones_encontradas = resultado_canciones["tracks"]["items"]
        
        # Si la página está vacía, no hay más resultados para este género/año
        if not canciones_encontradas:
            break

        # Procesar las canciones de la página
        for cancion in canciones_encontradas:
            # Controlar el límite de seguridad dentro del bucle
            if conteo_actual >= MAX_CANCIONES_POR_GENERO:
                break
                
            # Extracción de datos
            nombre_artista = cancion.get("artists", [{}])[0].get("name", "Desconocido")
            nombre_album = cancion.get("album", {}).get("name", "Desconocido")

            info = {
                "cancion": cancion.get("name"),
                "artista": nombre_artista,
                "album": nombre_album,
                "genero": genero,
                "año": año
            }
            lista_info_canciones.append(info)
            conteo_actual += 1

        # Si se alcanzó el límite dentro de la página, salir del bucle while
        if conteo_actual >= MAX_CANCIONES_POR_GENERO:
            print(f"    -> Límite de {MAX_CANCIONES_POR_GENERO} alcanzado.")
            break

        # Preparar el siguiente offset para la siguiente página
        offset += LIMIT_PER_REQUEST
        
        # Opcional: Para evitar sobrecargar la API de Spotify, puedes añadir una pequeña pausa:
        # time.sleep(0.5) 
        
    return lista_info_canciones

# --- 2. Bucle Principal (Mantiene la simplicidad) ---

generos = ["flamenco", "latin", "jazz", "rock"]
año = 2010
todas_las_canciones = []
canciones_por_genero = {}

print(f"Buscando todas las canciones para el año {año} (hasta {MAX_CANCIONES_POR_GENERO} por género)\n")

for genero in generos:
    print(f"--- 🎵 Procesando Género: {genero} ---")
    
    # La función ahora se encarga de la paginación internamente
    nuevas_canciones = obtener_canciones_por_genero_y_año(genero, año)
    
    # Acumular los resultados
    todas_las_canciones.extend(nuevas_canciones)
    
    canciones_por_genero[genero] = len(nuevas_canciones)
    print(f"Añadidas {len(nuevas_canciones)} canciones de '{genero}'\n")

# --- 3. Resumen y DataFrame ---

print("## 📊 Conteo Final de Canciones por Género")
for genero, contador in canciones_por_genero.items():
    print(f"- **{genero}**: {contador} canciones")

total_canciones = len(todas_las_canciones)
print(f"\n**Total de Canciones añadidas: {total_canciones}**")

# Crear DataFrame con los resultados
todas_las_canciones_df = pd.DataFrame(todas_las_canciones)

In [33]:
datos = spotify.search(q="genre:flamenco year:2010", type='track', limit=50, offset=0)

In [34]:
datos

{'tracks': {'href': 'https://api.spotify.com/v1/search?offset=0&limit=50&query=genre%3Aflamenco%20year%3A2010&type=track',
  'limit': 50,
  'next': None,
  'offset': 0,
  'previous': None,
  'total': 46,
  'items': [{'album': {'album_type': 'compilation',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0fqQJD6wePdVDxuPUVrLyX'},
       'href': 'https://api.spotify.com/v1/artists/0fqQJD6wePdVDxuPUVrLyX',
       'id': '0fqQJD6wePdVDxuPUVrLyX',
       'name': 'Carlos Montoya',
       'type': 'artist',
       'uri': 'spotify:artist:0fqQJD6wePdVDxuPUVrLyX'}],
     'available_markets': ['AR',
      'AU',
      'AT',
      'BE',
      'BO',
      'BR',
      'BG',
      'CA',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DK',
      'DO',
      'DE',
      'EC',
      'EE',
      'SV',
      'FI',
      'FR',
      'GR',
      'GT',
      'HN',
      'HK',
      'HU',
      'IS',
      'IE',
      'IT',
      'LV',
      'LT',
      'LU',
   

In [21]:
todas_las_canciones_df = pd.DataFrame(todas_las_canciones)  

In [22]:
todas_las_canciones_df.to_csv("")

,nombre,artista,album,genero,año
0,Caribe e Flamenco,Carlos Montoya,Flamenco Guitar,flamenco,2010
1,El Clavel Y La Rosa,Angelillo,Vintage Music No. 70 - LP: Angelillo,flamenco,2010
2,Fernanda,Angelillo,Vintage Music No. 70 - LP: Angelillo,flamenco,2010
3,Gloria Y Riqueza,Juanito Valderrama,Caminante Solitario,flamenco,2010
4,Vuelve A Vivir,Angelillo,Vintage Music No. 70 - LP: Angelillo,flamenco,2010
...,...,...,...,...,...
191,The Suburbs,Arcade Fire,The Suburbs (Deluxe),rock,2010
192,La Casa,Caramelos De Cianuro,Caramelos de Cianuro,rock,2010
193,Caribbean Queen (No More Love On the Run),Billy Ocean,The Very Best of Billy Ocean,rock,2010
194,Nada,Zoé,Colaboraciones España,rock,2010


# Pruebas funcion

In [78]:
def busqueda_spotify(generos, año):
    todas_las_canciones = []
    albumes_ya_vistos = set()  # Evita duplicados: un álbum puede aparecer al buscar varios artistas (ej. colaboraciones).
    datos_artistas = {}
    artistas_procesados = set()

    print("Buscando canciones del año", año)

    for genero in generos:
        print("Género:", genero)

        # 👉 ANOTACIÓN: "genre:" SOLO funciona al buscar artistas (type="artist")
        #    No funciona con álbumes ni canciones directamente.
        resultado_artistas = spotify.search(q="genre:" + genero, type="artist", limit=50)
        artistas = resultado_artistas["artists"]["items"]

        for artista in artistas:
            nombre_artista = artista["name"]

            # 👉 ANOTACIÓN: "artist:" y "year:" SÍ se pueden usar juntos
            #    al buscar álbumes (type="album") o canciones (type="track").
            #    Aquí buscamos álbumes del artista en el año indicado.
            busqueda = "artist:" + nombre_artista + " year:" + str(año)
            resultado_albumes = spotify.search(q=busqueda, type="album", limit=50)
            albumes = resultado_albumes["albums"]["items"]
            
            for album in albumes:
                id_album = album["id"]
                nombre_album = album["name"]
                
                # Si ya vimos este álbum, saltamos al siguiente
                if id_album in albumes_ya_vistos:
                    continue
                
                # Si no lo habíamos visto, lo marcamos como visto
                albumes_ya_vistos.add(id_album)

                # Pedimos las canciones del álbum (esto no usa anotaciones, es una llamada directa)
                resultado_canciones = spotify.album_tracks(id_album)
                canciones = resultado_canciones["items"]

                for cancion in canciones:
                    info = {
                        "nombre": cancion["name"],
                        "artista": nombre_artista,
                        "album": nombre_album,
                        "genero": genero,
                        "año": año
                    }
                    todas_las_canciones.append(info)

        print("  → Canciones de este género añadidas")
        print()

    # Mostramos un resumen de lo encontrado
    print("RESUMEN:")

    # Contamos las canciones de cada género una por una
    for genero in generos:
        contador = 0
        for cancion in todas_las_canciones:
            if cancion["genero"] == genero:
                contador = contador + 1
        print(f"- {genero}: {contador} canciones")

    print("Total de canciones encontradas:", len(todas_las_canciones))

    todas_las_canciones_df = pd.DataFrame(todas_las_canciones)  
    return todas_las_canciones_df

In [ ]:
canciones_2010 = busqueda_spotify(["classical", "latin", "jazz", "rock"], 2010)

Buscando canciones del año 2010
Género: classical
  → Canciones de este género añadidas

Género: latin
  → Canciones de este género añadidas

Género: jazz


Your application has reached a rate/request limit. Retry will occur after: 81796 s


In [77]:
todas_las_canciones_df.head()

,cancion,artista,genero,año
0,The Christmas Gift,Morat,latin,2010
1,Orphanization,Don Omar,latin,2010
2,El Duro,Don Omar,latin,2010
3,Hasta Abajo,Don Omar,latin,2010
4,Hooka,Don Omar,latin,2010


# Last FM

In [61]:
if not api_key_lastfm:
    print("ERROR: La variable de entorno 'API_KEY_LASTFM' no está configurada.")
else:
    print("API Key de Last.fm cargada con éxito.")

API Key de Last.fm cargada con éxito.


In [62]:
url_last_fm = ("http://ws.audioscrobbler.com/2.0/")

In [63]:
def busqueda_info_artista(nombre_artista, api_key_lastfm):
    artista_codificado = quote(nombre_artista) #esto sirve para que los espacios y caracteres especiales no interfieran con la url de la api
    
    url_last_fm = "http://ws.audioscrobbler.com/2.0/"
    params_info = {
                        'method': 'artist.getinfo',
                        'artist': artista_codificado,
                        'api_key': api_key_lastfm,
                        'format': 'json'
                    }
    try:
        response = requests.get(url_last_fm, params=params_info, timeout=10)
        response.raise_for_status()
        data = response.json()

        if "artist" in data: 
            artista_info = data['artist']
            bio_summary = artista_info.get('bio', {}).get('summary', '').split('<a href')[0].strip()
            return {
                        'bio_resumen': bio_summary,
                        'listeners': int(artista_info.get('stats', {}).get('listeners', 0)),
        }
        else:
            # Artista no encontrado por Last.fm
            return {'consulta_exitosa': False, 'error_lastfm': "No encontrado en Last.fm"}
        
    except requests.exceptions.RequestException as e:
        # Incluye HTTPError, ConnectionError, TimeoutError, etc.
        status_code = getattr(e.response, 'status_code', 'N/A')
        return {'consulta_exitosa': False, 'error_lastfm': f"Error API ({status_code}): {e}"}
    except Exception as e:
        return {'consulta_exitosa': False, 'error_lastfm': f"Error Procesamiento: {e}"}


if not api_key_lastfm:
    print("ERROR: La clave de la API de Last.fm no está configurada.")
else:
    artistas_unicos = todas_las_canciones_df['artista'].unique() #extrae los artistas sin repetir
    print(f"\nTotal de artistas únicos a consultar en Last.fm: {len(artistas_unicos)}")

    artistas_df = pd.DataFrame(artistas_unicos, columns=['artista']) #df temporal con los artistas sin repetir
    
    print("\nIniciando consultas a Last.fm...")
    
    #apply pasa el valor de la columna "artista" como primer argumento posicional. Con "args" se pasan el resto de argumentos.
    resultados_lastfm_serie = artistas_df['artista'].apply(
        busqueda_info_artista, 
        args=(api_key_lastfm,) 
    )

    # 4. Normalizar los resultados (convertir la Serie de Diccionarios a Columnas de DF)
    datos_lastfm_df = pd.json_normalize(resultados_lastfm_serie)
    datos_lastfm_df.insert(0, 'artista', artistas_unicos)
    
    print("Consultas a Last.fm terminadas y datos unidos al DataFrame.")


Total de artistas únicos a consultar en Last.fm: 84

Iniciando consultas a Last.fm...
Consultas a Last.fm terminadas y datos unidos al DataFrame.


In [64]:
datos_lastfm_df

,artista,bio_resumen,listeners
0,Morat,Morat is a Colombian band formed in the countr...,292486
1,Don Omar,"William Omar Landrón (born February 10, 1978 i...",1704144
2,Milo j,Camilo Joaquín Villarruel nació el 25 de octub...,243414
3,Beéle,"Brandon De Jesús López Orozco, mejor conocido ...",189238
4,Prince Royce,Geoffrey Royce Rojas was born and raised in th...,704524
...,...,...,...
79,Radiohead,Radiohead is an English rock band formed in Ab...,8012939
80,Queen,Queen are a British rock band formed in London...,7050785
81,Elton John,Sir Elton Hercules John (born Reginald Kenneth...,5079064
82,Maná,There is more than one artist with this name:\...,818108


# Creación BD

In [65]:
try:
    cnx = mysql.connector.connect(
        host= MYSQL_HOST,
        user= MYSQL_USER,
        password= MYSQL_PASSWORD,
    )
    print('Conexión exitosa')
except Error as e:
    print('Error al conectar:', e)

try:
    mycursor = cnx.cursor()
    query = "CREATE DATABASE IF NOT EXISTS MusicStream_db"
    mycursor.execute(query)
    print("Query exitosa")
except:
    print("Error.")

Conexión exitosa
Query exitosa


In [66]:
mycursor.execute("USE MusicStream_db")

# se añaden en varios execute porque a todas se les ha llamado query, por lo que reemplaza a la anterior y solo ejecuta la última válida

query = '''CREATE TABLE IF NOT EXISTS Canciones(
	ID_Cancion INT AUTO_INCREMENT,
    Nombre TEXT NOT NULL,
    Artista VARCHAR(45),
    Album TEXT,
    Genero VARCHAR(30),
    Lanzamiento YEAR,
    PRIMARY KEY (ID_Cancion)
);'''
mycursor.execute(query)

query = '''CREATE TABLE IF NOT EXISTS Artistas(
    ID_Artista INT AUTO_INCREMENT,
    Artista VARCHAR(70) UNIQUE NOT NULL,
    Info TEXT,
    Oyentes INT,
    PRIMARY KEY (ID_Artista)
);'''
mycursor.execute(query)

In [76]:
print(todas_las_canciones_df.columns)

Index(['cancion', 'artista', 'genero', 'año'], dtype='object')


In [74]:
# Query de inserción
mycursor.execute("USE MusicStream_db")
query_insert = """
INSERT INTO Canciones (Nombre, Artista, Album, Genero, Lanzamiento) 
VALUES (%s, %s, %s, %s, %s)
"""

try:
    df_corregido_SPOTIFY = todas_las_canciones_df.replace({np.nan: None, 'nan': None, 'NaN': None}) # corregimos valores nulos para que MySQL los entienda
    datos = df_corregido_SPOTIFY.values.tolist()

    mycursor.executemany(query_insert, datos)
    
    print(f"{mycursor.rowcount} registros insertados")
    cnx.commit() #indispensable para guardar los cambios y que se complete la petición hecha (sio da algún error antes del commit se quedan ambos programas en standby)

except Error as e:
    print("Error al insertar los datos:", e)
    cnx.rollback() # revertir petición en caso de error
    

Error al insertar los datos: Failed executing the operation; Not enough parameters for the SQL statement


In [75]:
# Query de inserción
mycursor.execute("USE MusicStream_db")
query_insert = """
INSERT INTO Canciones (Nombre, Artista, Album, Genero, Lanzamiento) 
VALUES (%s, %s, %s, %s, %s)
"""

try:
    df_corregido_SPOTIFY = todas_las_canciones_df.replace({np.nan: None, 'nan': None, 'NaN': None}) # corregimos valores nulos para que MySQL los entienda
    columnas_a_insertar = ['cancion', 'artista', 'album', 'genero', 'año']
    datos = df_corregido_SPOTIFY.values.tolist()

    mycursor.executemany(query_insert, datos)
    
    print(f"{mycursor.rowcount} registros insertados")
    cnx.commit() #indispensable para guardar los cambios y que se complete la petición hecha (sio da algún error antes del commit se quedan ambos programas en standby)

except Error as e:
    print("Error al insertar los datos:", e)
    cnx.rollback() # revertir petición en caso de error
    

Error al insertar los datos: Failed executing the operation; Not enough parameters for the SQL statement


In [71]:
# Query de inserción
mycursor.execute("USE MusicStream_db")
query_insert = """
INSERT INTO Artistas (Artista, Info, Oyentes) 
VALUES (%s, %s, %s)
"""

try:
    df_corregido_LASTFM = datos_lastfm_df.replace({np.nan: None, 'nan': None, 'NaN': None})
    datos = df_corregido_LASTFM.values.tolist()
    
    mycursor.executemany(query_insert, datos)
    
    print(f"{mycursor.rowcount} registros insertados")
    cnx.commit()

except Error as e:
    print("Error al insertar los datos:", e)
    cnx.rollback()

84 registros insertados


In [ ]:
cnx.close()